# Лабораторная работа 7. Классификация

Генерация данных

In [18]:
import numpy as np
import pandas as pd


rng = np.random.default_rng(146)
n = 300

df = pd.DataFrame({
"ВидПомещения": rng.choice([
"жилые помещения", "паркинг", "кладовые"],
size=n, p=[0.75, 0.15, 0.10]
),
"СледующийСтатус": rng.choice([
"Продана", "Свободна", "В резерве"],
size=n, p=[0.45, 0.45, 0.10]
),
"ИсточникБрони": rng.choice([
"МП", "ручная"], size=n, p=[0.6, 0.4]
),
"Город": rng.choice([
"Москва", "СПб", "Казань", "Екатеринбург"],
size=n, p=[0.4, 0.25, 0.2, 0.15]
),
"Тип": rng.choice([
"1к", "2к", "3к", "с"],
size=n, p=[0.3, 0.35, 0.25, 0.10]
),
"ПродаваемаяПлощадь": rng.normal(55, 15, size=n).clip(25, 120),
"ФактическаяСтоимостьПомещения": rng.normal(8_000_000, 2_500_000, size=n).clip(3_000_000, 20_000_000),
"СкидкаНаКвартиру": rng.choice([
0, 100_000, 200_000, -50_000, np.nan
], size=n, p=[0.45, 0.2, 0.15, 0.1, 0.1])
})

df.loc[rng.choice(df.index, size=25, replace=False), "ПродаваемаяПлощадь"] = np.nan


df.head()

,ВидПомещения,СледующийСтатус,ИсточникБрони,Город,Тип,ПродаваемаяПлощадь,ФактическаяСтоимостьПомещения,СкидкаНаКвартиру
0,паркинг,Продана,МП,СПб,2к,69.101528,4.108651e+06,0.0
1,жилые помещения,Продана,ручная,СПб,3к,37.594549,8.671242e+06,100000.0
2,жилые помещения,Свободна,ручная,Москва,с,57.189772,4.880254e+06,0.0
3,жилые помещения,Свободна,МП,Екатеринбург,2к,90.007870,5.732289e+06,0.0
4,кладовые,В резерве,ручная,Москва,2к,49.739853,7.358675e+06,0.0


Предварительная фильтрация

In [19]:
df = df[df["ВидПомещения"] == "жилые помещения"].copy()
df = df[df["СледующийСтатус"].isin(["Продана", "Свободна"])].copy()

Формирование целевой переменной

In [20]:
df["y"] = df["СледующийСтатус"].map({
"Продана": 1,
"Свободна": 0
})

df[["СледующийСтатус", "y"]]

,СледующийСтатус,y
1,Продана,1
2,Свободна,0
3,Свободна,0
5,Продана,1
6,Продана,1
...,...,...
293,Свободна,0
294,Свободна,0
297,Продана,1
298,Свободна,0


Удаление лишних столбцов и преобразование признака "Тип"

In [21]:
df = df.drop(columns=["ВидПомещения", "СледующийСтатус"])
def parse_rooms(x):
    if isinstance(x, str) and x.endswith("к"):
        return int(x[:-1])
    return np.nan


df["Тип"] = df["Тип"].apply(parse_rooms)
df[["Тип"]]

,Тип
1,3.0
2,NaN
3,2.0
5,3.0
6,2.0
...,...
293,2.0
294,3.0
297,1.0
298,2.0


Работа с пропусками

In [22]:
df["СкидкаНаКвартиру"] = df["СкидкаНаКвартиру"].fillna(0)

In [23]:
df["ПродаваемаяПлощадь"] = df["ПродаваемаяПлощадь"].fillna(df["ПродаваемаяПлощадь"].median())
df["Тип"] = df["Тип"].fillna(df["Тип"].median())

Переход от данных со словами к числам

In [24]:
df["ИсточникБрони"] = df["ИсточникБрони"].map({"МП": 1, "ручная": 0})
df = pd.get_dummies(df, columns=["Город"])

Feature engineering

In [25]:
df["Цена_за_м2"] = df["ФактическаяСтоимостьПомещения"] / df["ПродаваемаяПлощадь"]
df["Скидка_%"] = df["СкидкаНаКвартиру"] / df["ФактическаяСтоимостьПомещения"]

Отделяем данные

In [26]:
y = df["y"]
X = df.drop(columns=["y"])

### Приведение всех признаков к одному диапазону.
алгоритмы на расстояниях (KNN) чувствительны к масштабу.

In [27]:
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

##### Разделение данных

In [28]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
X_scaled, y, test_size=0.3, random_state=146
)

#### KNN

In [29]:
from sklearn.neighbors import KNeighborsClassifier


knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


#### Decision Tree

In [30]:
from sklearn.tree import DecisionTreeClassifier


tree = DecisionTreeClassifier(random_state=146)
tree.fit(X_train, y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,146
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [31]:
from sklearn.metrics import precision_score, recall_score, f1_score


pred = knn.predict(X_test)


precision_score(y_test, pred)
recall_score(y_test, pred)
f1_score(y_test, pred)

0.5172413793103449

# Не забудьте рассказать что значат показатели